In [5]:
from binance.client import Client
import pandas as pd
import ta
import time
import gate_api
import requests
from gate_api.exceptions import ApiException, GateApiException
import numpy as np

# Binance API client oluştur
client = Client(api_key='IzUtb0XbG6w0hui8y2SdmnCaHADkO2Y6sMBM3wNxuIbSmTK6uJmNle2BSSzU9lpA', api_secret='HNVVVkE5N3Af7uMzDOyrpD6YFObRvphUqN4vmEuSNdRgU2JAw4u8D4fw4KMfSkjV')


# Sembol ve zaman dilimini ayarla
SYMBOL = 'WIF_USDT'
BINANCE_SYMBOL = 'WIFUSDT'

# Son 100 mumu al
klines = client.get_historical_klines(BINANCE_SYMBOL, '1m', '30 minutes ago UTC')

# Son iki mumu seç
#last_two_klines = klines[-5:]

last_two_klines = klines[-5:]

# Değişim oranlarını hesapla
change_percentages = [((float(kline[4]) - float(kline[1])) / float(kline[1])) * 100 for kline in last_two_klines]

# Verileri bir DataFrame'e atama
data = {"Mum": range(1, len(change_percentages) + 1),
        "Değişim Oranı": change_percentages}
df = pd.DataFrame(data)
df['Toplam Değişim'] = df['Değişim Oranı'].cumsum()
son_degisim_df = df.iloc[[-1]].copy()


In [6]:
from binance.client import Client
import time
import gate_api
import requests
from gate_api.exceptions import ApiException, GateApiException
import numpy as np
import pandas as pd

# API anahtarları
BINANCE_API_KEY = 'key'
BINANCE_API_SECRET = 'secret'
GATEIO_API_KEY = 'key'
GATEIO_API_SECRET = 'secret'

# Sabitler
SYMBOL = 'BTC_USDT'
BINANCE_SYMBOL = 'BTCUSDT'
SLEEP_DURATION = 3 # saniye
position_open = False

# Binance ve Gate.io için API client nesnelerini oluşturma
client = Client(api_key=BINANCE_API_KEY, api_secret=BINANCE_API_SECRET)
configuration = gate_api.Configuration(key=GATEIO_API_KEY, secret=GATEIO_API_SECRET)
gateio_api_client = gate_api.ApiClient(configuration)
gateio_spot_api = gate_api.SpotApi(gateio_api_client)

def get_balance_difference_gateio(api_client, spot_api, symbol):
    current_price = float(spot_api.list_tickers(currency_pair=symbol)[0].last)
    return current_price

def get_balance_difference_binance(client, symbol):
    current_price = client.get_symbol_ticker(symbol=symbol)
    return current_price['price']

SIMULATION_BALANCE_USDT = 10.0  # Başlangıç simülasyon bakiyesi (USDT)
SIMULATION_BTC = 0  # Başlangıçta BTC miktarı

azalma_durumu = None

try:
    while True:
        #gateio ve binance de priceları ve balanceyi hesapla!
        price_gateio= get_balance_difference_gateio(gateio_api_client, gateio_spot_api, SYMBOL)
        price_binance = get_balance_difference_binance(client, BINANCE_SYMBOL)
        
        #pricelar str geliyor floata çevirdik.
        price_gateio_f = float(price_gateio)
        price_binance_f = float(price_binance)
        
        # Son 100 mumu al
        klines = client.get_historical_klines(BINANCE_SYMBOL, '1m', '100 minutes ago UTC')
        last_two_klines = klines[-1:]
        
        #Hareketli Ortalama Hesaplama
        last_three_klines = klines[-1:]
        last_5_closes = [float(kline[4]) for kline in last_three_klines]
        moving_average = sum(last_5_closes) / len(last_5_closes)

        # Değişim oranlarını hesapla
        change_percentages = [((float(kline[4]) - float(kline[1])) / float(kline[1])) * 100 for kline in last_two_klines]
        
        data = {"Mum": range(1, len(change_percentages) + 1),
        "Değişim Oranı": change_percentages}
        df = pd.DataFrame(data)
        df['Toplam Değişim'] = df['Değişim Oranı'].cumsum()
        
        son_degisim_df = df.iloc[[-1]].copy()
        son_degisim_orani = son_degisim_df['Değişim Oranı'].iloc[0]
        son_toplam_orani = son_degisim_df['Toplam Değişim'].iloc[0]

        # mum en yüksek kapanış fiyatları çekme
        latest_candle = klines[-1]
        open_price = float(latest_candle[1])  # Açılış fiyatı
        close = float(latest_candle[4])  # Kapanış fiyatı

        # Tüm verilerin birleştirilerek yazdırılması
        print(f"Fiyat:{price_binance_f:.4f}, Son Degisim: {son_degisim_orani:.4f}, Toplam Oran: {son_toplam_orani:.4f}, Moving: {moving_average:.4f}")
        
        #Alım Sinyal
        #if not position_open and azalma_durumu is not None and azalma_durumu < son_degisim_orani and close > open_price:
        if not position_open and azalma_durumu is not None and azalma_durumu < son_toplam_orani:
        #if not position_open and moving_average + 0.0100 > price_binance_f :
            position_open = True
            bought_btc = 10 / price_binance_f
            SIMULATION_BALANCE_USDT -= 10
            SIMULATION_BTC += bought_btc
            test = 10.03  / price_binance_f
            print(f"Alım yapıldı: {bought_btc} BTC, Yeni bakiye: {SIMULATION_BALANCE_USDT} USDT, BTC: {SIMULATION_BTC}, test:{test}")
        
        # Satım sinyali
        elif position_open:
            sold_usdt = SIMULATION_BTC * price_binance_f
            fee = sold_usdt * 0.002
            net_usdt = sold_usdt - fee
            print(net_usdt)
            if net_usdt >= 10.01:
                position_open = False
                SIMULATION_BALANCE_USDT += net_usdt
                SIMULATION_BTC = 0
                print(f"Satım yapıldı: {sold_usdt} USDT, Yeni bakiye: {SIMULATION_BALANCE_USDT} USDT, BTC: {SIMULATION_BTC}")
        
        azalma_durumu = son_toplam_orani
        
        time.sleep(SLEEP_DURATION)
        
except KeyboardInterrupt:
    print("Döngü manuel olarak durduruldu.")
except Exception as e:
    print(f"Bir hata oluştu: {e}")


Fiyat:66609.3300, Son Degisim: -0.0025, Toplam Oran: -0.0025, Moving: 66609.3300
Fiyat:66609.3300, Son Degisim: 0.0035, Toplam Oran: 0.0035, Moving: 66613.3400
Alım yapıldı: 0.00015012911854840757 BTC, Yeni bakiye: 0.0 USDT, BTC: 0.00015012911854840757, test:0.00015057950590405278
Fiyat:66617.8000, Son Degisim: 0.0045, Toplam Oran: 0.0045, Moving: 66613.9900
9.981269050446839
Fiyat:66585.2800, Son Degisim: -0.0452, Toplam Oran: -0.0452, Moving: 66580.9400
9.976396615909513
Fiyat:66588.0000, Son Degisim: -0.0346, Toplam Oran: -0.0346, Moving: 66588.0000
9.97680415040956
Fiyat:66590.2100, Son Degisim: -0.0312, Toplam Oran: -0.0312, Moving: 66590.2100
9.977135272190848
Fiyat:66588.0000, Son Degisim: -0.0345, Toplam Oran: -0.0345, Moving: 66588.0100
9.97680415040956
Fiyat:66588.0000, Son Degisim: -0.0346, Toplam Oran: -0.0346, Moving: 66588.0000
9.97680415040956
Fiyat:66599.2000, Son Degisim: -0.0165, Toplam Oran: -0.0165, Moving: 66600.0000
9.978482233645048
Fiyat:66589.6300, Son Degisim:

Fiyat:66591.9900, Son Degisim: 0.0114, Toplam Oran: 0.0114, Moving: 66592.0000
9.977401967562203
Fiyat:66607.8000, Son Degisim: 0.0351, Toplam Oran: 0.0351, Moving: 66607.8000
9.979770761843724
Fiyat:66610.2000, Son Degisim: 0.0387, Toplam Oran: 0.0387, Moving: 66610.1900
9.980130351108471
Fiyat:66605.3400, Son Degisim: 0.0314, Toplam Oran: 0.0314, Moving: 66605.3400
9.979402182847359
Fiyat:66589.8000, Son Degisim: 0.0081, Toplam Oran: 0.0081, Moving: 66589.8000
9.977073842358122
Fiyat:66589.8100, Son Degisim: 0.0081, Toplam Oran: 0.0081, Moving: 66589.8000
9.977075340646723
Fiyat:66586.0100, Son Degisim: 0.0024, Toplam Oran: 0.0024, Moving: 66586.0100
9.976505990977541
Fiyat:66586.0100, Son Degisim: 0.0024, Toplam Oran: 0.0024, Moving: 66586.0000
9.976505990977541
Fiyat:66584.4100, Son Degisim: 0.0000, Toplam Oran: 0.0000, Moving: 66584.4100
9.976266264801044
Fiyat:66570.5600, Son Degisim: -0.0208, Toplam Oran: -0.0208, Moving: 66570.5600
9.97419113508573
Fiyat:66589.8100, Son Degisim